# TP1 - Adrian Jose Zapater Reig

# Ejercicio 2: HIVE.
Ejemplo de uso de HIVE sobre 2 sets de datos.

## Diseño

En primer lugar vamos a subir los ficheros a HDFS, una base de datos de HIVE y unas tablas auxiliares externas para cargar los datos. Usaremos OpenCSVSerde para que HIVE sea capaz de detectar las columnas separadas por ',' y el carácter '"' para separador de texto.

Para poder hacer uso de la función COALESCE, necesitamos que las cadenas vacías se interpreten como NULL. La propiedad serialization.null.format nos permite elegir qué carácter o combinación de carácteres se interpretará como NULL. Como no es compatible con OpenCSVSerde usaremos una tabla auxiliar e insertaremos los datos en la tabla definitiva.

Puesto que HIVE es capaz de inferir el esquema de la tabla a partir de los datos, podemos usar la sentencia CREATE TABLE AS SELECT... para crear las tablas internar y ahorrarnos el trabajo de especificarlo explícitamente.

Para crear las vistas, nos apoyaremos en la función COALESCE() para comprobar que las columnas NO son nulas y/o dar valores por defecto.

Para calcular la media de cada final, tendremos que averiguar cuantas columnas tienen valor distinto de NULL, sumarlas (sumar 0 si la columna es NULL) y dividir por el número total de columnas distintas de NULL. Nos apoyaremos en la cláusula CASE WHEN para aplicar la lógica como si fuera un IF ELSE.
Si todas las columnas son NULL, devolveremos 0 para que la salída sea homogénea.

Para resolver las cuestiones finales nos apoyaremos en la sentencia de ordenanción ORDER BY para listar los resultados en orden de mas a menor y usaremos LIMIT para devolver los 10 resultados que se piden.
Para la última cuestión, tendremos que cruzar la información de ambos ficheros con la sentencia JOIN y filtrar con la sentencia WHERE para obtener el resultado que buscamos. Las queries son autoexplicativas.

#### NOTAS:
En caso de que se quisiera obtener únicamente el nombre del país en las primeras 3 cuestiones, se tendría que añadir un nivel mas a la query:

SELECT a.country_name FROM ($QUERY_DE_CUESTION) a

Debemos asignar un alias a la query de segundo nivel para que HIVE sea capaz de procesarla.

Directorio de trabajo local: /media/notebook/tp1-notebooks/ejer_2_hive

Directorio de trabajo HDFS:  /user/uned/databases/ejer2_hive

Directorio de datos local:   /media/notebook/datos

In [1]:
! mkdir -p /media/notebook/tp1-notebooks/ejer_2_hive

In [2]:
import os
os.chdir("/media/notebook/tp1-notebooks/ejer_2_hive")

In [3]:
! pwd

/media/notebook/tp1-notebooks/ejer_2_hive


In [4]:
! hadoop fs -mkdir -p /user/uned/databases/ejer2_hive

In [5]:
! hadoop fs -ls /user/uned/databases/

Found 1 items
drwxr-xr-x   - root supergroup          0 2019-11-12 20:13 /user/uned/databases/ejer2_hive


In [6]:
! hadoop fs -mkdir -p /user/uned/databases/ejer2_hive/external/TablaExternaGDP
! hadoop fs -mkdir -p /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion

In [7]:
! hadoop fs -ls /user/uned/databases/ejer2_hive/external/

Found 2 items
drwxr-xr-x   - root supergroup          0 2019-11-12 20:13 /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion
drwxr-xr-x   - root supergroup          0 2019-11-12 20:13 /user/uned/databases/ejer2_hive/external/TablaExternaGDP


## Creacion de la tablas
Creamos 2 tablas externas con LOCATION = a la ruta HDFS donde hemos dejado los ficheros con datos para que HIVE los lea al acceder a la tabla.

Creamos 2 tablas internas desde una select a la externa para que HIVE use los datos cargados en la externa.

Nos topamos con un problema: una de las columnas tiene '"' como separador de campo de texto y contiene  ',' dentro de dicho campo. Para que HIVE sea capaz de leer dicho campo como un solo campo usaremos el SerDe OpenCSVSerde según la documentacion de confluence: https://cwiki.apache.org/confluence/display/Hive/CSV+Serde

In [8]:
%%writefile create_aux_tables.hql

--Creamos la BD que contendra las tablas
CREATE DATABASE IF NOT EXISTS ejer2_hive
COMMENT 'BD para el ejercicio 2 del TP1'
LOCATION '/user/uned/databases/ejer2_hive';

--Borramos las tablas si existen.
DROP TABLE IF EXISTS ejer2_hive.TablaInternaGDP;
DROP TABLE IF EXISTS ejer2_hive.TablaExternaGDP;
DROP TABLE IF EXISTS ejer2_hive.TablaInternaEscolarizacion;
DROP TABLE IF EXISTS ejer2_hive.TablaExternaEscolarizacion;

--Creamos tabla aux en la BD ejer2_hive
CREATE EXTERNAL TABLE ejer2_hive.auxGDP ( Country_Name STRING,
Country_Code STRING,
Indicator_Name STRING,
Indicator_Code STRING,
`1960` DECIMAL(38,15),`1961` DECIMAL(38,15),`1962` DECIMAL(38,15),`1963` DECIMAL(38,15),
`1964` DECIMAL(38,15),`1965` DECIMAL(38,15),`1966` DECIMAL(38,15),`1967` DECIMAL(38,15),
`1968` DECIMAL(38,15),`1969` DECIMAL(38,15),`1970` DECIMAL(38,15),`1971` DECIMAL(38,15),
`1972` DECIMAL(38,15),`1973` DECIMAL(38,15),`1974` DECIMAL(38,15),`1975` DECIMAL(38,15),
`1976` DECIMAL(38,15),`1977` DECIMAL(38,15),`1978` DECIMAL(38,15),`1979` DECIMAL(38,15),
`1980` DECIMAL(38,15),`1981` DECIMAL(38,15),`1982` DECIMAL(38,15),`1983` DECIMAL(38,15),
`1984` DECIMAL(38,15),`1985` DECIMAL(38,15),`1986` DECIMAL(38,15),`1987` DECIMAL(38,15),
`1988` DECIMAL(38,15),`1989` DECIMAL(38,15),`1990` DECIMAL(38,15),`1991` DECIMAL(38,15),
`1992` DECIMAL(38,15),`1993` DECIMAL(38,15),`1994` DECIMAL(38,15),`1995` DECIMAL(38,15),
`1996` DECIMAL(38,15),`1997` DECIMAL(38,15),`1998` DECIMAL(38,15),`1999` DECIMAL(38,15),
`2000` DECIMAL(38,15),`2001` DECIMAL(38,15),`2002` DECIMAL(38,15),`2003` DECIMAL(38,15),
`2004` DECIMAL(38,15),`2005` DECIMAL(38,15),`2006` DECIMAL(38,15),`2007` DECIMAL(38,15),
`2008` DECIMAL(38,15),`2009` DECIMAL(38,15),`2010` DECIMAL(38,15),`2011` DECIMAL(38,15),
`2012` DECIMAL(38,15),`2013` DECIMAL(38,15),`2014` DECIMAL(38,15),`2015` DECIMAL(38,15),
`2016` DECIMAL(38,15)
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   "separatorChar" = ",",
   "quoteChar"     = "\"",
   "escapeChar"    = "\\"
)
STORED AS TEXTFILE
LOCATION "/user/uned/databases/ejer2_hive/external/TablaExternaGDP_aux";

--Creamos aux en la BD ejer2_hive
CREATE EXTERNAL TABLE ejer2_hive.auxEsc ( Country_Name STRING,
Country_Code STRING,
Indicator_Name STRING,
Indicator_Code STRING,
`1960` DECIMAL(38,15),`1961` DECIMAL(38,15),`1962` DECIMAL(38,15),`1963` DECIMAL(38,15),
`1964` DECIMAL(38,15),`1965` DECIMAL(38,15),`1966` DECIMAL(38,15),`1967` DECIMAL(38,15),
`1968` DECIMAL(38,15),`1969` DECIMAL(38,15),`1970` DECIMAL(38,15),`1971` DECIMAL(38,15),
`1972` DECIMAL(38,15),`1973` DECIMAL(38,15),`1974` DECIMAL(38,15),`1975` DECIMAL(38,15),
`1976` DECIMAL(38,15),`1977` DECIMAL(38,15),`1978` DECIMAL(38,15),`1979` DECIMAL(38,15),
`1980` DECIMAL(38,15),`1981` DECIMAL(38,15),`1982` DECIMAL(38,15),`1983` DECIMAL(38,15),
`1984` DECIMAL(38,15),`1985` DECIMAL(38,15),`1986` DECIMAL(38,15),`1987` DECIMAL(38,15),
`1988` DECIMAL(38,15),`1989` DECIMAL(38,15),`1990` DECIMAL(38,15),`1991` DECIMAL(38,15),
`1992` DECIMAL(38,15),`1993` DECIMAL(38,15),`1994` DECIMAL(38,15),`1995` DECIMAL(38,15),
`1996` DECIMAL(38,15),`1997` DECIMAL(38,15),`1998` DECIMAL(38,15),`1999` DECIMAL(38,15),
`2000` DECIMAL(38,15),`2001` DECIMAL(38,15),`2002` DECIMAL(38,15),`2003` DECIMAL(38,15),
`2004` DECIMAL(38,15),`2005` DECIMAL(38,15),`2006` DECIMAL(38,15),`2007` DECIMAL(38,15),
`2008` DECIMAL(38,15),`2009` DECIMAL(38,15),`2010` DECIMAL(38,15),`2011` DECIMAL(38,15),
`2012` DECIMAL(38,15),`2013` DECIMAL(38,15),`2014` DECIMAL(38,15),`2015` DECIMAL(38,15),
`2016` DECIMAL(38,15)
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
   "separatorChar" = ",",
   "quoteChar"     = "\"",
   "escapeChar"    = "\\"
)
STORED AS TEXTFILE
LOCATION "/user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion_aux"


Overwriting create_aux_tables.hql


In [9]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f create_aux_tables.hql

2019-11-12 20:13:35,330 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
s: jdbc:hive2://localhost:10000/default> --Creamos la BD que contendra las tabla 
e: jdbc:hive2://localhost:10000/default> CREATE DATABASE IF NOT EXISTS ejer2_hiv 
': jdbc:hive2://localhost:10000/default> COMMENT 'BD para el ejercicio 2 del TP1 
ve';dbc:hive2://localhost:10000/default> LOCATION '/user/uned/databases/ejer2_hi 
INFO  : Compiling command(queryId=hive_20191112201313_dd53342f-be97-4e13-bce2-4250501afa66): CREATE DATABASE IF NOT EXISTS ejer2_hive
COMMENT 'BD para el ejercicio 2 del TP1'
LOCATION '/user/uned/databases/ejer2_hive'
INFO  : Semantic Analysis Co

In [10]:
#Copiamos los datos a HDFS...
! hadoop fs -put /media/notebook/datos/API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv \
/user/uned/databases/ejer2_hive/external/TablaExternaGDP_aux
! hadoop fs -ls /user/uned/databases/ejer2_hive/external/TablaExternaGDP_aux

Found 1 items
-rw-r--r--   1 root supergroup     205586 2019-11-12 20:13 /user/uned/databases/ejer2_hive/external/TablaExternaGDP_aux/API_NY.GDP.MKTP.CD_DS2_en_csv_v2.csv


In [11]:
#Copiamos los datos a HDFS...
! hadoop fs -put /media/notebook/datos/API_SE.PRM.UNER.FE_DS2_en_csv_v2.csv \
/user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion_aux
! hadoop fs -ls /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion_aux

Found 1 items
-rw-r--r--   1 root supergroup      63371 2019-11-12 20:13 /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion_aux/API_SE.PRM.UNER.FE_DS2_en_csv_v2.csv


In [12]:
%%writefile create_external_tables.hql

CREATE EXTERNAL TABLE ejer2_hive.TablaExternaGDP ( Country_Name STRING,
Country_Code STRING,
Indicator_Name STRING,
Indicator_Code STRING,
`1960` DECIMAL(38,15),`1961` DECIMAL(38,15),`1962` DECIMAL(38,15),`1963` DECIMAL(38,15),
`1964` DECIMAL(38,15),`1965` DECIMAL(38,15),`1966` DECIMAL(38,15),`1967` DECIMAL(38,15),
`1968` DECIMAL(38,15),`1969` DECIMAL(38,15),`1970` DECIMAL(38,15),`1971` DECIMAL(38,15),
`1972` DECIMAL(38,15),`1973` DECIMAL(38,15),`1974` DECIMAL(38,15),`1975` DECIMAL(38,15),
`1976` DECIMAL(38,15),`1977` DECIMAL(38,15),`1978` DECIMAL(38,15),`1979` DECIMAL(38,15),
`1980` DECIMAL(38,15),`1981` DECIMAL(38,15),`1982` DECIMAL(38,15),`1983` DECIMAL(38,15),
`1984` DECIMAL(38,15),`1985` DECIMAL(38,15),`1986` DECIMAL(38,15),`1987` DECIMAL(38,15),
`1988` DECIMAL(38,15),`1989` DECIMAL(38,15),`1990` DECIMAL(38,15),`1991` DECIMAL(38,15),
`1992` DECIMAL(38,15),`1993` DECIMAL(38,15),`1994` DECIMAL(38,15),`1995` DECIMAL(38,15),
`1996` DECIMAL(38,15),`1997` DECIMAL(38,15),`1998` DECIMAL(38,15),`1999` DECIMAL(38,15),
`2000` DECIMAL(38,15),`2001` DECIMAL(38,15),`2002` DECIMAL(38,15),`2003` DECIMAL(38,15),
`2004` DECIMAL(38,15),`2005` DECIMAL(38,15),`2006` DECIMAL(38,15),`2007` DECIMAL(38,15),
`2008` DECIMAL(38,15),`2009` DECIMAL(38,15),`2010` DECIMAL(38,15),`2011` DECIMAL(38,15),
`2012` DECIMAL(38,15),`2013` DECIMAL(38,15),`2014` DECIMAL(38,15),`2015` DECIMAL(38,15),
`2016` DECIMAL(38,15)
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY '|'
STORED AS TEXTFILE
LOCATION "/user/uned/databases/ejer2_hive/external/TablaExternaGDP"
TBLPROPERTIES('serialization.null.format'='');

INSERT OVERWRITE TABLE ejer2_hive.TablaExternaGDP SELECT * FROM ejer2_hive.auxGDP;


--Creamos TablaExternaEscolarizacion en la BD ejer2_hive
CREATE EXTERNAL TABLE ejer2_hive.TablaExternaEscolarizacion ( Country_Name STRING,
Country_Code STRING,
Indicator_Name STRING,
Indicator_Code STRING,
`1960` DECIMAL(38,15),`1961` DECIMAL(38,15),`1962` DECIMAL(38,15),`1963` DECIMAL(38,15),
`1964` DECIMAL(38,15),`1965` DECIMAL(38,15),`1966` DECIMAL(38,15),`1967` DECIMAL(38,15),
`1968` DECIMAL(38,15),`1969` DECIMAL(38,15),`1970` DECIMAL(38,15),`1971` DECIMAL(38,15),
`1972` DECIMAL(38,15),`1973` DECIMAL(38,15),`1974` DECIMAL(38,15),`1975` DECIMAL(38,15),
`1976` DECIMAL(38,15),`1977` DECIMAL(38,15),`1978` DECIMAL(38,15),`1979` DECIMAL(38,15),
`1980` DECIMAL(38,15),`1981` DECIMAL(38,15),`1982` DECIMAL(38,15),`1983` DECIMAL(38,15),
`1984` DECIMAL(38,15),`1985` DECIMAL(38,15),`1986` DECIMAL(38,15),`1987` DECIMAL(38,15),
`1988` DECIMAL(38,15),`1989` DECIMAL(38,15),`1990` DECIMAL(38,15),`1991` DECIMAL(38,15),
`1992` DECIMAL(38,15),`1993` DECIMAL(38,15),`1994` DECIMAL(38,15),`1995` DECIMAL(38,15),
`1996` DECIMAL(38,15),`1997` DECIMAL(38,15),`1998` DECIMAL(38,15),`1999` DECIMAL(38,15),
`2000` DECIMAL(38,15),`2001` DECIMAL(38,15),`2002` DECIMAL(38,15),`2003` DECIMAL(38,15),
`2004` DECIMAL(38,15),`2005` DECIMAL(38,15),`2006` DECIMAL(38,15),`2007` DECIMAL(38,15),
`2008` DECIMAL(38,15),`2009` DECIMAL(38,15),`2010` DECIMAL(38,15),`2011` DECIMAL(38,15),
`2012` DECIMAL(38,15),`2013` DECIMAL(38,15),`2014` DECIMAL(38,15),`2015` DECIMAL(38,15),
`2016` DECIMAL(38,15)
)
ROW FORMAT DELIMITED FIELDS TERMINATED BY '|'
STORED AS TEXTFILE
LOCATION "/user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion"
TBLPROPERTIES('serialization.null.format'='');

INSERT OVERWRITE TABLE ejer2_hive.TablaExternaEscolarizacion SELECT * FROM ejer2_hive.auxEsc;

-- Borramos las tablas aux
DROP TABLE ejer2_hive.auxGDP;
DROP TABLE ejer2_hive.auxEsc;


Overwriting create_external_tables.hql


In [13]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f create_external_tables.hql

2019-11-12 20:13:53,190 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
xternaGDP ( Country_Name STRING,default> CREATE EXTERNAL TABLE ejer2_hive.TablaE 
0: jdbc:hive2://localhost:10000/default> Country_Code STRING,
0: jdbc:hive2://localhost:10000/default> Indicator_Name STRING,
0: jdbc:hive2://localhost:10000/default> Indicator_Code STRING,
,15),`1962` DECIMAL(38,15),`1963` DECIMAL(3960` DECIMAL(38,15),`1961` DECIMAL(38 8,15),
,15),`1966` DECIMAL(38,15),`1967` DECIMAL(38,15),ECIMAL(38,15),`1965` DECIMAL(38 
,15),`1970` DECIMAL(38,15),`1971` DECIMAL(38,15),ECIMAL(38,15),`1969` DECIMAL(38 
,15),`1974` DECIMAL(38,15),`1975` DECIMAL(38,15),E

In [14]:
#Borramos los datos aux de HDFS...
! hadoop fs -rm -r -f /user/uned/databases/ejer2_hive/external/TablaExternaGDP_aux
! hadoop fs -rm -r -f /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion_aux

Deleted /user/uned/databases/ejer2_hive/external/TablaExternaGDP_aux
Deleted /user/uned/databases/ejer2_hive/external/TablaExternaEscolarizacion_aux


In [15]:
%%writefile create_internal_tables.hql

--Creamos TablaInternaGDP en la BD ejer2_hive
CREATE TABLE ejer2_hive.TablaInternaGDP AS SELECT * FROM ejer2_hive.TablaExternaGDP;

--Creamos TablaInternaEscolarizacion en la BD ejer2_hive
CREATE TABLE ejer2_hive.TablaInternaEscolarizacion AS SELECT * FROM ejer2_hive.TablaExternaEscolarizacion;

Overwriting create_internal_tables.hql


In [16]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f create_internal_tables.hql

2019-11-12 20:14:31,150 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
2_hivec:hive2://localhost:10000/default> --Creamos TablaInternaGDP en la BD ejer 
 AS SELECT * FROlocalhost:10000/default> CREATE TABLE ejer2_hive.TablaInternaGDP M ejer2_hive.TablaExternaGDP;
INFO  : Compiling command(queryId=hive_20191112201414_fea343c4-fc3a-4553-9491-9881cd746eee): CREATE TABLE ejer2_hive.TablaInternaGDP AS SELECT * FROM ejer2_hive.TablaExternaGDP
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:tablaexternagdp.country_name, type:string, comment:null), FieldSchema(name:tablaexternagdp.country_

## Creamos las vistas
Las vistas tiene 4 campos:

    1-name -- Nombre del pais
    2-code -- Codigo del pais
    3-avg_country_1 -- Media por pais entre 2000 y 2009 .
    4-avg_country_2 -- Media por pais entre 2010 y actualidad.

Primero comprobamos que no existen paises repetidos.
Solo lo comprobamos para un set de tablas (en este caso internas) porque contienen los mismos datos.

In [17]:
%%writefile count_countries.hql

-- Ambas queries deben dar lo mismo: 264
SELECT count(country_code)  FROM ejer2_hive.TablaInternaEscolarizacion;
SELECT count(distinct country_code)  FROM ejer2_hive.TablaInternaEscolarizacion;

-- Ambas queries deben dar lo mismo: 264
SELECT count(country_code)  FROM ejer2_hive.TablaInternaGDP;
SELECT count(distinct country_code)  FROM ejer2_hive.TablaInternaGDP;



Overwriting count_countries.hql


In [18]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f count_countries.hql

2019-11-12 20:14:55,429 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
4: jdbc:hive2://localhost:10000/default> -- Ambas queries deben dar lo mismo: 26 
hive.TablaInternaEscolarizacion;default> SELECT count(country_code)  FROM ejer2_ 
INFO  : Compiling command(queryId=hive_20191112201414_3eb6dcbf-17c9-4977-bca8-64e2402e0caa): SELECT count(country_code)  FROM ejer2_hive.TablaInternaEscolarizacion
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:_c0, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=hive_20191112201414_3eb6dcbf-17c9-4977-bca8-64

In [19]:
%%writefile create_views.hql

-- Borramos las vistas si existen.
DROP VIEW IF EXISTS ejer2_hive.VistaTablaExternaGDP;
DROP VIEW IF EXISTS ejer2_hive.VistaTablaExternaEscolarizacion;
DROP VIEW IF EXISTS ejer2_hive.VistaTablaInternaGDP;
DROP VIEW IF EXISTS ejer2_hive.VistaTablaInternaEscolarizacion;


-- Creamos las 4 vistas.
CREATE VIEW ejer2_hive.VistaTablaExternaGDP AS    
        SELECT country_name as name, country_code as code, 
            CASE
                WHEN COALESCE(`2000`, `2001`, `2002`, `2003`, `2004`, `2005`, `2006`, `2007`, `2008`, `2009`) IS NOT NULL THEN
                    (COALESCE(`2000`,0) + COALESCE(`2001`,0) + COALESCE(`2002`,0) + COALESCE(`2003`,0) + COALESCE(`2004`,0) + 
                     COALESCE(`2005`,0) + COALESCE(`2006`,0) + COALESCE(`2007`,0) + COALESCE(`2008`,0) + COALESCE(`2009`,0)) / 
                        (CASE WHEN `2000` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2001` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2002` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2003` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2004` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2005` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2006` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2007` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2008` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2009` IS NULL THEN 0 ELSE 1 END)
                ELSE 0
            END as avg_country_1,
            CASE
                WHEN COALESCE(`2010`, `2011`, `2012`, `2013`, `2014`, `2015`, `2016`) IS NOT NULL THEN
                    (COALESCE(`2010`,0) + COALESCE(`2011`,0) + COALESCE(`2012`,0) + COALESCE(`2013`,0) + COALESCE(`2014`,0) + 
                     COALESCE(`2015`,0) + COALESCE(`2016`,0)) / 
                        (CASE WHEN `2010` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2011` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2012` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2013` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2014` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2015` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2016` IS NULL THEN 0 ELSE 1 END)
                ELSE 0
            END as avg_country_2
            FROM ejer2_hive.TablaExternaGDP;

CREATE VIEW ejer2_hive.VistaTablaExternaEscolarizacion AS    
        SELECT country_name as name, country_code as code, 
            CASE
                WHEN COALESCE(`2000`, `2001`, `2002`, `2003`, `2004`, `2005`, `2006`, `2007`, `2008`, `2009`) IS NOT NULL THEN
                    (COALESCE(`2000`,0) + COALESCE(`2001`,0) + COALESCE(`2002`,0) + COALESCE(`2003`,0) + COALESCE(`2004`,0) + 
                     COALESCE(`2005`,0) + COALESCE(`2006`,0) + COALESCE(`2007`,0) + COALESCE(`2008`,0) + COALESCE(`2009`,0)) / 
                        (CASE WHEN `2000` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2001` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2002` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2003` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2004` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2005` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2006` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2007` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2008` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2009` IS NULL THEN 0 ELSE 1 END)
                ELSE 0
            END as avg_country_1,
            CASE
                WHEN COALESCE(`2010`, `2011`, `2012`, `2013`, `2014`, `2015`, `2016`) IS NOT NULL THEN
                    (COALESCE(`2010`,0) + COALESCE(`2011`,0) + COALESCE(`2012`,0) + COALESCE(`2013`,0) + COALESCE(`2014`,0) + 
                     COALESCE(`2015`,0) + COALESCE(`2016`,0)) / 
                        (CASE WHEN `2010` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2011` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2012` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2013` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2014` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2015` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2016` IS NULL THEN 0 ELSE 1 END)
                ELSE 0
            END as avg_country_2
            FROM ejer2_hive.TablaExternaEscolarizacion;

            
CREATE VIEW ejer2_hive.VistaTablaInternaGDP AS    
        SELECT country_name as name, country_code as code, 
            CASE
                WHEN COALESCE(`2000`, `2001`, `2002`, `2003`, `2004`, `2005`, `2006`, `2007`, `2008`, `2009`) IS NOT NULL THEN
                    (COALESCE(`2000`,0) + COALESCE(`2001`,0) + COALESCE(`2002`,0) + COALESCE(`2003`,0) + COALESCE(`2004`,0) + 
                     COALESCE(`2005`,0) + COALESCE(`2006`,0) + COALESCE(`2007`,0) + COALESCE(`2008`,0) + COALESCE(`2009`,0)) / 
                        (CASE WHEN `2000` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2001` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2002` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2003` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2004` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2005` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2006` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2007` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2008` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2009` IS NULL THEN 0 ELSE 1 END)
                ELSE 0
            END as avg_country_1,
            CASE
                WHEN COALESCE(`2010`, `2011`, `2012`, `2013`, `2014`, `2015`, `2016`) IS NOT NULL THEN
                    (COALESCE(`2010`,0) + COALESCE(`2011`,0) + COALESCE(`2012`,0) + COALESCE(`2013`,0) + COALESCE(`2014`,0) + 
                     COALESCE(`2015`,0) + COALESCE(`2016`,0)) / 
                        (CASE WHEN `2010` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2011` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2012` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2013` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2014` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2015` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2016` IS NULL THEN 0 ELSE 1 END)
                ELSE 0
            END as avg_country_2
            FROM ejer2_hive.TablaInternaGDP;
    
    
    
    CREATE VIEW ejer2_hive.VistaTablaInternaEscolarizacion AS    
        SELECT country_name as name, country_code as code, 
            CASE
                WHEN COALESCE(`2000`, `2001`, `2002`, `2003`, `2004`, `2005`, `2006`, `2007`, `2008`, `2009`) IS NOT NULL THEN
                    (COALESCE(`2000`,0) + COALESCE(`2001`,0) + COALESCE(`2002`,0) + COALESCE(`2003`,0) + COALESCE(`2004`,0) + 
                     COALESCE(`2005`,0) + COALESCE(`2006`,0) + COALESCE(`2007`,0) + COALESCE(`2008`,0) + COALESCE(`2009`,0)) / 
                        (CASE WHEN `2000` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2001` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2002` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2003` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2004` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2005` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2006` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2007` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2008` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2009` IS NULL THEN 0 ELSE 1 END)
                ELSE 0
            END as avg_country_1,
            CASE
                WHEN COALESCE(`2010`, `2011`, `2012`, `2013`, `2014`, `2015`, `2016`) IS NOT NULL THEN
                    (COALESCE(`2010`,0) + COALESCE(`2011`,0) + COALESCE(`2012`,0) + COALESCE(`2013`,0) + COALESCE(`2014`,0) + 
                     COALESCE(`2015`,0) + COALESCE(`2016`,0)) / 
                        (CASE WHEN `2010` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2011` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2012` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2013` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2014` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2015` IS NULL THEN 0 ELSE 1 END +
                         CASE WHEN `2016` IS NULL THEN 0 ELSE 1 END)
                ELSE 0
            END as avg_country_2
            FROM ejer2_hive.TablaInternaEscolarizacion;
    

Overwriting create_views.hql


In [20]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f create_views.hql

2019-11-12 20:16:05,816 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
0: jdbc:hive2://localhost:10000/default> -- Borramos las vistas si existen.
laExternaGDP;://localhost:10000/default> DROP VIEW IF EXISTS ejer2_hive.VistaTab 
INFO  : Compiling command(queryId=hive_20191112201616_e8724f6d-dd9f-48f9-a353-48623544c31b): DROP VIEW IF EXISTS ejer2_hive.VistaTablaExternaGDP
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=hive_20191112201616_e8724f6d-dd9f-48f9-a353-48623544c31b); Time taken: 0.019 seconds
INFO  : Concurrency mode is disa

## Consultas sobre las cuestiones del ejercicio
Este apartado contiene las 4 consultas del ejercicio 4.

o ¿Cuáles son los 10 paises que tienen mayor GDP en el año 2010?

o ¿Cuáles son los 10 paises que tienen mayor GDP medio en la decada de 2010?

o ¿Cuáles son los 10 paises en los que hay más niñas sin escolarizar en 2010?

o ¿Cuál fue el GDP en 2015 del pais que ese año tuvo 8339 niñas sin escolarizar?


### 1- ¿Cuáles son los 10 paises que tienen mayor GDP en el año 2010?

In [21]:
%%writefile consulta_1.hql

SELECT country_name, country_code, `2010` 
FROM ejer2_hive.TablaExternaGDP 
ORDER BY `2010` DESC 
LIMIT 10;

Overwriting consulta_1.hql


In [22]:
#+----------------------------+---------------+-------------------+--+
#|        country_name        | country_code  |       2010        |
#+----------------------------+---------------+-------------------+--+
#| World                      | WLD           | 65906150720610.2  |
#| High income                | HIC           | 45187761296453.4  |
#| OECD members               | OED           | 44588338316642.4  |
#| Post-demographic dividend  | PST           | 42396023961852.1  |
#| IDA & IBRD total           | IBT           | 21396210564680.8  |
#| Europe & Central Asia      | ECS           | 20905471026131.4  |
#| Low & middle income        | LMY           | 20662142675904.1  |
#| Middle income              | MIC           | 20431462486141.4  |
#| IBRD only                  | IBD           | 19954364474673.9  |
#| European Union             | EUU           | 16977855795546.3  |
#+----------------------------+---------------+-------------------+--+

In [23]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f consulta_1.hql

2019-11-12 20:16:10,046 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 3ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
0` jdbc:hive2://localhost:10000/default> SELECT country_name, country_code, `201 
0: jdbc:hive2://localhost:10000/default> FROM ejer2_hive.TablaExternaGDP 
0: jdbc:hive2://localhost:10000/default> ORDER BY `2010` DESC 
0: jdbc:hive2://localhost:10000/default> LIMIT 10;
INFO  : Compiling command(queryId=hive_20191112201616_d5469a5c-ece0-4565-932f-a3c0038f74a4): SELECT country_name, country_code, `2010`
FROM ejer2_hive.TablaExternaGDP 
ORDER BY `2010` DESC 
LIMIT 10
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:

### 2- ¿Cuáles son los 10 paises que tienen mayor GDP medio en la decada de 2010?

In [24]:
%%writefile consulta_2.hql

SELECT name, code, avg_country_2 
FROM ejer2_hive.VistaTablaExternaGDP 
ORDER BY avg_country_2 DESC 
LIMIT 10;

Overwriting consulta_2.hql


In [25]:
#+----------------------------+-------+--------------------------------------+--+
#|            name            | code  |            avg_country_2             |
#+----------------------------+-------+--------------------------------------+--+
#| World                      | WLD   | 74256883825525.1                     |
#| High income                | HIC   | 48311785768532.75714285714285714286  |
#| OECD members               | OED   | 47411797217976.1                     |
#| Post-demographic dividend  | PST   | 44999128842638.42857142857142857143  |
#| IDA & IBRD total           | IBT   | 26684814058083.84285714285714285714  |
#| Low & middle income        | LMY   | 25902637587038.92857142857142857143  |
#| Middle income              | MIC   | 25591493690219.18571428571428571429  |
#| IBRD only                  | IBD   | 24849753693843.94285714285714285714  |
#| Europe & Central Asia      | ECS   | 21946805108940.92857142857142857143  |
#| East Asia & Pacific        | EAS   | 20710612806670.92857142857142857143  |
#+----------------------------+-------+--------------------------------------+--+

In [26]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f consulta_2.hql

2019-11-12 20:16:29,345 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
0: jdbc:hive2://localhost:10000/default> SELECT name, code, avg_country_2 
0: jdbc:hive2://localhost:10000/default> FROM ejer2_hive.VistaTablaExternaGDP 
0: jdbc:hive2://localhost:10000/default> ORDER BY avg_country_2 DESC 
0: jdbc:hive2://localhost:10000/default> LIMIT 10;
INFO  : Compiling command(queryId=hive_20191112201616_050a9537-48ac-49d9-a952-abb09f88db56): SELECT name, code, avg_country_2
FROM ejer2_hive.VistaTablaExternaGDP 
ORDER BY avg_country_2 DESC 
LIMIT 10
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSche

### 3- ¿Cuáles son los 10 paises en los que hay más niñas sin escolarizar en 2010?

In [27]:
%%writefile consulta_3.hql

SELECT country_name, country_code, `2010` 
FROM ejer2_hive.TablaExternaEscolarizacion 
ORDER BY `2010` DESC 
LIMIT 10;



Overwriting consulta_3.hql


In [28]:
#+---------------------------------------------+---------------+-----------+--+
#|                country_name                 | country_code  |   2010    |
#+---------------------------------------------+---------------+-----------+--+
#| World                                       | WLD           | 31787794  |
#| IDA & IBRD total                            | IBT           | 31512640  |
#| IDA total                                   | IDA           | 24823300  |
#| Middle income                               | MIC           | 19319244  |
#| Sub-Saharan Africa                          | SSF           | 19238428  |
#| Sub-Saharan Africa (IDA & IBRD countries)   | TSS           | 19238428  |
#| Sub-Saharan Africa (excluding high income)  | SSA           | 19238388  |
#| Pre-demographic dividend                    | PRE           | 18051788  |
#| IDA only                                    | IDX           | 15958392  |
#| Lower middle income                         | LMC           | 15397692  |
#+---------------------------------------------+---------------+-----------+--+

In [29]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f consulta_3.hql

2019-11-12 20:16:47,947 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 4ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> 
0` jdbc:hive2://localhost:10000/default> SELECT country_name, country_code, `201 
ion dbc:hive2://localhost:10000/default> FROM ejer2_hive.TablaExternaEscolarizac 
0: jdbc:hive2://localhost:10000/default> ORDER BY `2010` DESC 
0: jdbc:hive2://localhost:10000/default> LIMIT 10;
INFO  : Compiling command(queryId=hive_20191112201616_17fba6a1-3c61-4a8d-84b6-16fe65210a7b): SELECT country_name, country_code, `2010`
FROM ejer2_hive.TablaExternaEscolarizacion 
ORDER BY `2010` DESC 
LIMIT 10
INFO  : Semantic Analysis Completed
INFO  : Returning Hive schema: Schema(fieldSchemas

### 4-  ¿Cuál fue el GDP en 2015 del pais que ese año tuvo 8339 niñas sin escolarizar?

In [30]:
%%writefile consulta_4.hql
SELECT gdp.`2015` 
FROM ejer2_hive.TablaExternaGDP gdp 
JOIN ejer2_hive.TablaExternaEscolarizacion esc 
    ON gdp.country_code = esc.country_code 
WHERE esc.`2015`= 8339;


Overwriting consulta_4.hql


In [31]:
#+-------------------+--+
#|     gdp.2015      |
#+-------------------+--+
#| 357949199759.586  |
#+-------------------+--+

In [32]:
! beeline -u "jdbc:hive2://localhost:10000/default" -f consulta_4.hql

2019-11-12 20:17:10,812 WARN  [main] mapreduce.TableMapReduceUtil: The hbase-prefix-tree module jar containing PrefixTreeCodec is not present.  Continuing without it.
scan complete in 2ms
Connecting to jdbc:hive2://localhost:10000/default
Connected to: Apache Hive (version 1.1.0-cdh5.7.0)
Driver: Hive JDBC (version 1.1.0-cdh5.7.0)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000/default> SELECT gdp.`2015` 
0: jdbc:hive2://localhost:10000/default> FROM ejer2_hive.TablaExternaGDP gdp 
ion esc hive2://localhost:10000/default> JOIN ejer2_hive.TablaExternaEscolarizac 
ode dbc:hive2://localhost:10000/default>     ON gdp.country_code = esc.country_c 
0: jdbc:hive2://localhost:10000/default> WHERE esc.`2015`= 8339;
INFO  : Compiling command(queryId=hive_20191112201717_a2dc0ab8-aede-4409-9a25-6dd1f933ad21): SELECT gdp.`2015`
FROM ejer2_hive.TablaExternaGDP gdp 
JOIN ejer2_hive.TablaExternaEscolarizacion esc 
    ON gdp.country_code = esc.country_code 
WHERE esc